<div align="center"; span style="color:#336699"><b><h2>Minicurso pyForTraCC </h2></b></div>
<div align="center"; span style="color:#336699"><b><h3>2. Rastreio Operacional em Tempo Real com Dados do Satélite GOES-19</h3></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
   <sup>Autores: <a href="https://www.linkedin.com/in/helvecio-leal/"> Helvécio B. Leal Neto, <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><t>&nbsp;</t> 
   <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="mailto:fortracc.project@inpe.br">fortracc.project@inpe.br</a>
    <br/><br/>
    Última Atualização: Set 7, 2025
</div>

<br/>

<div style="text-align: center; margin: 0 auto; width: 80ch;">
<b>Resumo.</b> Este notebook faz parte do minicurso de introdução ao <a href="https://github.com/fortracc/pyfortracc">pyfortracc</a> e demonstra um exemplo completo de rastreamento operacional usando dados reais do satélite GOES-19. O tutorial abrange desde o download automático de dados de temperatura de brilho do sensor ABI até análises espaciais avançadas com integração de dados administrativos brasileiros. Inclui rastreamento de sistemas convectivos de mesoescala (SCM), aplicação de métodos de correção vetorial, visualizações cartográficas interativas e identificação de municípios com SCM ativos - fornecendo uma base sólida para aplicações de nowcasting meteorológico em tempo real.
</div>

<div align="center">

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fortracc/pyfortracc/blob/main/examples/WORCAP-Minicourse/2_RealTime_Tracking/2_RealTime_Tracking.ipynb) 

</div>
<br/>

### Roteiro
 [1. Instalação](#install)<br>
 [2. Dados de Entrada](#input)<br>
 [3. Função de Leitura](#data)<br>
 [4. Parâmetros (Name_list)](#namelist)<br>
 [5. Rotina de Rastreamento](#track)<br>
 [6. Tabela de Rastreamento](#tracktable)<br>
 [7. Visualização de Rastreamento](#visualization)<br>
 [8. Nowcasting / Real-Time Tracking](#nowcasting)<br>
    [8.1. Análise Espacial - Municípios Afetados](#spatial-analysis)<br>
 [9. Conclusão](#conclusion)<br>

##### 1. Instalação <a class="anchor" id="install"></a>

Para instalar a biblioteca pyFortraCC, você pode usar o pip. Execute o seguinte comando:

In [ ]:
# Comando para instalação da ultima versão estável do pyfortracc
!python -m pip install -qqq -U pyfortracc > /dev/null 2>&1 && echo "✅ pyfortracc instalado com sucesso!" || echo "❌ Erro na instalação"

Após a instalação, você pode importar a biblioteca e verificar a versão instalada:

In [ ]:
# Importa o pyfortracc e mostra a versão instalada
import pyfortracc
print('pyFortracc version', pyfortracc.__version__)

##### 2. Dados de Entrada <a class="anchor" id="input"></a>

Neste exemplo, utilizaremos dados reais de satélite do GOES-19. Usaremos o pacote `goesgcp` para baixar as imagens mais recentes do satélite GOES-19 diretamente do Google Cloud Platform (GCP).

Para instalar o pacote `goesgcp`, execute o comando abaixo:

In [ ]:
# Instalação do goesgcp do PyPI
!python -m pip install -qqq -U goesgcp > /dev/null 2>&1 && echo "✅ goesgcp instalado com sucesso!" || echo "❌ Erro na instalação do goesgcp"

Após a instalação do `goesgcp`, podemos baixar as 10 imagens mais recentes do satélite GOES-19 e reprojetar os dados para uma grade regular com resolução de 0.045 graus (aproximadamente 5 km). Os dados são recortados para a região do Brasil.

**Importante**: O download pode levar alguns minutos dependendo da sua conexão com a internet.

In [ ]:
# Remove qualquer diretório input/ anterior
!rm -rf input/

# Download das 10 imagens mais recentes do GOES-19
# --recent 10: baixa as 10 imagens mais recentes
# --output "input/": salva no diretório input/
# --resolution 0.045: resolução de ~5km
# --lat_min -35 --lat_max 5: limites de latitude (Sul para Norte)
# --lon_min -80 --lon_max -30: limites de longitude (Oeste para Leste)
!goesgcp --recent 10 --output "input/" --resolution 0.045 --lat_min -35 --lat_max 5 --lon_min -80 --lon_max -30

##### 3. Função de Leitura <a class="anchor" id="data"></a>

A função `read_function` é responsável por ler os dados dos arquivos de entrada. Neste caso, os dados são arquivos NetCDF contendo dados de temperatura de brilho do canal 13 (infravermelho) do sensor ABI do GOES-19.

A função retorna um array numpy contendo os dados de temperatura de brilho:

In [ ]:
# Função para ler os dados dos arquivos NetCDF baixados
import xarray as xr
import glob

def read_function(path):
    """
    Lê dados de temperatura de brilho de arquivos NetCDF do GOES-19
    
    Args:
        path (str): Caminho para o arquivo NetCDF
        
    Returns:
        numpy.array: Array 2D com dados de temperatura de brilho (invertido verticalmente)
    """
    ds = xr.open_dataset(path)
    # Retorna os dados de CMI (Cloud and Moisture Imagery) invertidos verticalmente
    # para corresponder à orientação geográfica padrão (Norte no topo)
    return ds['CMI'].data[::-1]

Para definir os limites espaciais dos dados, utilizamos a função `read_function` para ler a primeira imagem e extrair os valores de latitude e longitude:

In [ ]:
# Define os limites espaciais do domínio
files = glob.glob('input/*.nc')

# Verifica se arquivos foram baixados
if not files:
    print("Nenhum arquivo encontrado! Execute primeiro o download dos dados.")
else:
    # Abre o primeiro arquivo para extrair coordenadas
    ds = xr.open_dataset(files[0])
    
    # Extrai os limites de longitude e latitude do domínio
    lon_min = float(ds['lon'].min().values)
    lon_max = float(ds['lon'].max().values)
    lat_min = float(ds['lat'].min().values)
    lat_max = float(ds['lat'].max().values)
    
    print(f"Limites do domínio:")
    print(f"  Longitude: {lon_min:.2f}° a {lon_max:.2f}°")
    print(f"  Latitude: {lat_min:.2f}° a {lat_max:.2f}°")

##### 4. Parâmetros (Name_list) <a class="anchor" id="namelist"></a>

O dicionário `name_list` contém os parâmetros necessários para o algoritmo de rastreamento. Para este exemplo, utilizaremos parâmetros associados ao rastreamento de sistemas convectivos de mesoescala (SCM).

Rastreamos os SCM usando dados de temperatura de brilho do sensor ABI e definimos:
- **Limiares de temperatura**: de 235 K a 200 K (das temperaturas mais quentes para as mais frias)
- **Área mínima do sistema**: 400 pixels (aproximadamente 10.000 km²) para o primeiro limiar
- **Área mínima seguinte**: 100 pixels (aproximadamente 2.500 km²) para o segundo limiar

**Explicação dos principais parâmetros**:

In [ ]:
# Define o dicionário name_list com os parâmetros de rastreamento
name_list = {}

# === PARÂMETROS OBRIGATÓRIOS ===
name_list['input_path'] = 'input/'  # Diretório dos arquivos de entrada
name_list['output_path'] = 'output/'  # Diretório dos arquivos de saída

# Limiares para rastreamento (temperaturas de brilho em Kelvin)
# 235K: detecta núcleos convectivos moderados
# 200K: detecta núcleos convectivos intensos (topos de nuvens muito frias)
name_list['thresholds'] = [235, 200]

# Tamanho mínimo de cada cluster individual (em pixels)
# 400 pixels ≈ 10.000 km² para T ≤ 235K
# 100 pixels ≈ 2.500 km² para T ≤ 200K
name_list['min_cluster_size'] = [400, 100]

# Operador para segmentação baseado nos limiares
# '<=' significa: temperatura MENOR OU IGUAL aos limiares
name_list['operator'] = '<='

# Padrão de timestamp dos nomes dos arquivos GOES-19
# Formato: OR_ABI-L2-CMIPF-M6C13_G19_s20241234567...
name_list['timestamp_pattern'] = 'OR_ABI-L2-CMIPF-M6C13_G19_s%Y%j%H%M'
name_list['pattern_position'] = [0, 38]  # Posição do timestamp no nome do arquivo

# Intervalo de tempo entre imagens (em minutos)
name_list['delta_time'] = 10

# === PARÂMETROS ESPACIAIS ===
name_list['lon_min'] = lon_min  # Longitude mínima do domínio
name_list['lon_max'] = lon_max  # Longitude máxima do domínio
name_list['lat_min'] = lat_min  # Latitude mínima do domínio
name_list['lat_max'] = lat_max  # Latitude máxima do domínio

# === PARÂMETROS DE CORREÇÃO VETORIAL ===
# (conforme artigo: https://doi.org/10.3390/rs14215408)
name_list['spl_correction'] = True  # Correção em eventos de divisão (splitting)
name_list['mrg_correction'] = True  # Correção em eventos de fusão (merging)
name_list['inc_correction'] = True  # Correção usando vetores de núcleo interno
name_list['opt_correction'] = True  # Correção usando método de fluxo óptico
name_list['elp_correction'] = True  # Correção usando método de elipse
name_list['validation'] = True     # Validação dos métodos de correção

print("Parâmetros configurados com sucesso!")
print(f"Domínio: {lon_min:.1f}°-{lon_max:.1f}°W, {lat_min:.1f}°-{lat_max:.1f}°S")
print(f"Limiares: {name_list['thresholds']} K")
print(f"Áreas mínimas: {name_list['min_cluster_size']} pixels")

##### 5. Rotina de Rastreamento <a class="anchor" id="track"></a>

A função `track` é a função principal para rastrear os sistemas convectivos usando o pacote pyfortracc. 

Esta função executa todo o processo de rastreamento, incluindo:
1. **Feature Extraction**: Extrai características dos objetos detectados, como área, centroide, intensidade máxima, etc.
2. **Spatial Operations**: Realiza operações espaciais para identificar e conectar objetos entre frames.
3. **Linking**: Conecta objetos entre frames consecutivos baseado em proximidade e características
4. **Concat**: Concatena os resultados em uma tabela de rastreamento.

**Importante**: Esta operação pode levar vários minutos dependendo do número de imagens e da complexidade dos sistemas.

In [ ]:
# Importa a função principal de rastreamento do pyfortracc
from pyfortracc import track

In [ ]:
# Executa o rastreamento dos sistemas convectivos
print("Iniciando o rastreamento...")
# Chama a função principal de rastreamento
track(name_list, read_function)

print("Rastreamento concluído com sucesso!")
print("Os resultados foram salvos no diretório:", name_list['output_path'])

##### 6. Tabela de Rastreamento <a class="anchor" id="tracktable"></a>

A saída principal da função `track` é a **tabela de rastreamento**, que contém informações detalhadas sobre cada sistema convectivo rastreado ao longo do tempo.

A tabela de rastreamento em multiplos arquivos Parquet (um para cada frame) contém colunas como:

* **`timestamp`** (datetime64[us]): Informação temporal do cluster.  
* **`uid`** (float64): Identificador único do cluster.  
* **`iuid`** (float64): Identificador único interno do cluster.  
* **`threshold_level`** (int64): Nível do limiar.  
* **`threshold`** (float64): Limiar específico.  
* **`status`** (object): Status da entidade (NEW, CON, SPL, MRG, SPL/MRG)  
* **`u_, v_`** (float64): Componentes do vetor, além dos vetores oriundos dos métodos de correção, exemplo: `u_opt, v_opt` (método optical flow).
* **`inside_clusters`** (object): Número de clusters internos.  
* **`size`** (int64): Tamanho do cluster em pixels.  
* **`min, mean, max, std`** (float64): Estatísticas descritivas.  
* **`delta_time`** (timedelta64[us]): Variação temporal.  
* **`file`** (object): Nome do arquivo associado.  
* **`array_y, array_x`** (object): Coordenadas do array do cluster.  
* **`vector_field`** (object): Campo vetorial associado.  
* **`trajectory`** (object): Trajetória do cluster.  
* **`geometry`** (object): Representação geométrica da fronteira do cluster.  
* **`lifetime`** (int64): Vida útil do cluster em minutos.

Uma forma eficiente para carregar o rastreio é utilizando a biblioteca `duckdb`, que é eficiente para carregar os dados de rastreio e também permite a realização de consultas SQL com extensão espacial.

In [ ]:
# Importa duckdb para consulta eficiente de arquivos Parquet
import duckdb

# Conecta ao banco de dados em memória
con = duckdb.connect(database=':memory:', read_only=False)

# Instala e carrega a extensão espacial
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

Após criar a conexão e instalar a extensão espacial, carregamos os dados de rastreamento em uma tabela chamada `tracking`, que pode ser consultada usando SQL.

In [ ]:
# Carrega os dados de rastreamento em uma tabela chamada 'tracking'
con.execute(f"""
    CREATE OR REPLACE TABLE tracking AS
    SELECT * FROM parquet_scan('output/track/trackingtable/*.parquet',
                     union_by_name=True)
""")

Com a tabela carregada, você pode realizar consultas SQL para analisar os dados de rastreamento. Por exemplo, para exibir a quantidade de sistemas rastreados para o período de rastreio, você pode executar a seguinte consulta SQL:

In [ ]:
# Exibe o total de sistemas rastreados no período de rastreio
# A coluna 'uid' identifica cada sistema rastreado para o limiar 235K e pega a data de início e fim de rastreio

query = """
SELECT
    COUNT(DISTINCT uid) AS total_sistemas_235K,
    MIN(timestamp) AS inicio_rastreio,
    MAX(timestamp) AS fim_rastreio
FROM tracking
WHERE threshold = 235
"""

# Executa a consulta e armazena o resultado em uma lista
result = con.execute(query).fetchall()

print("Total de sistemas rastreados (T ≤ 235K):", result[0][0])
print("Período de rastreio: De", result[0][1], "até", result[0][2])

##### 7. Visualização de Rastreamento <a class="anchor" id="visualization"></a>

O pyfortracc oferece poderosas ferramentas de visualização para examinar os resultados do rastreamento. A função `plot_animation` permite criar animações que mostram:

- **Evolução temporal**: Como os sistemas se movem e evoluem ao longo do tempo
- **Dados de temperatura**: Visualização dos dados de infravermelho de fundo
- **Sistemas rastreados**: Contornos e identificação dos sistemas detectados
- **Trajetórias**: Caminhos percorridos pelos sistemas ao longo do tempo
- **Informações do sistema**: UID, tempo de vida e outras características

Para criar a animação, precisamos configurar os parâmetros de visualização, incluindo dimensões do gráfico, projeção e extensão geográfica:

In [ ]:
# Importa as bibliotecas necessárias para visualização
from pyfortracc import plot_animation
import numpy as np

In [ ]:
# Cria animação do rastreamento
print("Gerando animação do rastreamento...")

# Chama a função de plotagem de animação
plot_animation(
    # === CONFIGURAÇÕES BÁSICAS ===
    read_function=read_function,                    # Função de leitura dos dados
    figsize=(8, 6),                               # Tamanho da figura (largura, altura)
    name_list=name_list,                           # Dicionário com parâmetros
    
    # === CONFIGURAÇÕES TEMPORAIS ===
    start_timestamp=result[0][1],  # Timestamp inicial
    end_timestamp=result[0][2],    # Timestamp final

    # === CONFIGURAÇÕES DE VISUALIZAÇÃO ===
    info_col_name=False,                           # Não mostra nome das colunas de info
    cbar_title='Temperatura de Brilho (K)',       # Título da barra de cores
    trajectory=True,                               # Mostra trajetórias dos sistemas
    smooth_trajectory=True,                        # Suaviza as trajetórias
    
    # === CONFIGURAÇÕES DE CORES ===
    cmap='turbo',                                  # Mapa de cores (turbo: azul=frio, vermelho=quente)
    min_val=160,                                   # Valor mínimo para escala de cores (K)
    max_val=235,                                   # Valor máximo para escala de cores (K)
    nan_value=235,                                 # Valor para pixels sem dados
    nan_operation=np.greater_equal,                # Operação para identificar NaN
    
    # === CONFIGURAÇÕES DOS CONTORNOS E INFORMAÇÕES ===
    bound_color='blue',                            # Cor dos contornos dos sistemas
    info_cols=['uid'],                             # Colunas de informação a exibir
    
    # === CONFIGURAÇÕES DE PERFORMANCE ===
    parallel=False                                 # Processamento serial (mais estável)
)

##### 8. Nowcasting / Real-Time Tracking <a class="anchor" id="nowcasting"></a>

Uma aplicação interessante do pyfortracc é o rastreamento em tempo real (real-time tracking) de sistemas atmosféricos. Isso permite monitorar e prever a evolução de fenômenos como frentes frias, tempestades e outros eventos climáticos em alta resolução temporal.

Para complementar a visualização do rastreio, podemos detectar em quais municípios os sistemas rastreados estão localizados. Para isso, utilizamos o pacote `geobr` para carregar as geometrias dos municípios do Brasil.

Primeiro, instalaremos o pacote `geobr`:

In [ ]:
# Comando para instalação da biblioteca geobr
!python -m pip install -qqq -U geobr > /dev/null 2>&1 && echo "✅ geobr instalado com sucesso!" || echo "❌ Erro na instalação"

Caso a instalação seja bem-sucedida, você verá a mensagem "✅ geobr instalado com sucesso!". Em seguida podemos importar a biblioteca:

In [ ]:
# Importa a biblioteca geobr
import geobr

Em seguida, Carregamos todas as geometrias dos municípios do Brasil usando o `geobr`:

In [ ]:
# Carrega todas as geometrias dos municípios do Brasil para o ano de 2020
br_municipios = geobr.read_municipality(code_muni="all", year=2020)

Abaixo estão as primeiras linhas do DataFrame `br_municipios`, que contém informações sobre os municípios brasileiros, incluindo códigos, nomes e geometrias:

In [ ]:
# Mostra as primeiras linhas do DataFrame com os municípios
br_municipios.head()

Agora com os dados de municípios carregados, podemos realizar uma análise espacial para identificar quais municípios estão com SCMs ativos. Isso pode ser feito utilizando operações de interseção espacial entre as geometrias dos sistemas rastreados e as geometrias dos municípios.

#### 8.1. Preparação dos Dados Geométricos

Para realizar operações espaciais no DuckDB, precisamos converter as geometrias dos municípios para o formato WKT (Well-Known Text) e registrá-las no banco de dados:

In [ ]:
# Converte a geometria de br_municipios para WKT (string)
br_municipios["geometry"] = br_municipios["geometry"].apply(lambda g: g.wkt)

# Registra no DuckDB
con.register("br_municipios", br_municipios)

#### 8.2. Interseção Espacial - Clusters vs. Municípios

Agora realizaremos uma consulta SQL que utiliza a função espacial `ST_Intersects` para identificar quais municípios estão sendo interceptados pelos sistemas convectivos rastreados. 

A consulta abaixo:
- Seleciona dados do último timestamp disponível
- Filtra apenas sistemas com temperatura ≤ 235K
- Realiza interseção entre geometrias dos clusters e municípios
- Retorna informações dos municípios afetados e características dos sistemas

In [ ]:
# Query usando ST_GeomFromText para ambas as tabelas
sql = f"""
SELECT 
    m.name_muni,
    m.name_state,
    t.uid,
    t.min,
    t.lifetime,
    ST_AsText(ST_GeomFromText(t.geometry)) AS cluster_geometry,
    ST_AsText(ST_GeomFromText(m.geometry)) AS municipio_geometry
FROM tracking t
JOIN br_municipios m
    ON ST_Intersects(
        ST_GeomFromText(t.geometry),
        ST_GeomFromText(m.geometry)
    )
WHERE t.threshold = 235 AND t.timestamp = '{result[0][2]}'
"""

df_municipios_ativos = con.execute(sql).fetchdf()

#### 8.3. Análise Estatística dos Municípios com SCM ativos <a class="anchor" id="spatial-analysis"></a>

Com os dados da interseção espacial, podemos analisar estatísticas interessantes sobre os municípios que estão com SCMs ativos:

In [ ]:
print(f"Total de municípios ativos por sistemas com T ≤ 235K em {result[0][2]}: {df_municipios_ativos['name_muni'].nunique()}")
print(f"Munícipios com sistemas convectivos mais duradouros (maior lifetime):\n\n {df_municipios_ativos.sort_values(by='lifetime', ascending=False).head(10)[['name_muni', 'name_state', 'lifetime', 'min']]}")

#### 8.4. Preparação de Dados Geoespaciais para Visualização

Para criar visualizações cartográficas dos resultados, precisamos converter os dados em GeoDataFrames (estruturas geoespaciais do GeoPandas) e carregar as geometrias dos estados brasileiros:

In [ ]:
import geopandas as gpd

# Geometria dos clusters rastreados
gdf_municipios_ativos = gpd.GeoDataFrame(
    df_municipios_ativos,
    geometry=gpd.GeoSeries.from_wkt(df_municipios_ativos["municipio_geometry"]),
    crs="EPSG:4326"
)

# Geometria dos municípios
gdf_clusters_ativos = gpd.GeoDataFrame(
    df_municipios_ativos,
    geometry=gpd.GeoSeries.from_wkt(df_municipios_ativos["cluster_geometry"]),
    crs="EPSG:4326"
)

# Carrega as geometrias dos estados do Brasil para o ano de 2020
estados_brasil = geobr.read_state(year=2020)

#### 8.5. Mapa dos Municípios Afetados por Sistemas Convectivos

Importamos as bibliotecas necessárias para criar mapas com projeção cartográfica adequada:

In [ ]:
# Importa bibliotecas para visualização
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy import feature as cfeature

Finalmente, criamos um mapa que mostra:

- **Base cartográfica**: Relevo, oceanos, fronteiras e estados brasileiros
- **Municípios afetados**: Municípios interceptados pelos sistemas rastreados
- **Clusters convectivos**: Contornos em vermelho dos sistemas detectados (T ≤ 235K)
- **Contexto geográfico**: Localização precisa sobre o território brasileiro

Este tipo de visualização é essencial para:
- **Monitoramento meteorológico**: Identificação rápida de áreas de risco
- **Comunicação de alertas**: Informação clara para autoridades locais
- **Análise espacial**: Compreensão da distribuição geográfica dos fenômenos

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_title(f'Municípios com sistemas convectivos ativos (T ≤ 235K) em {result[0][2]} GMT', fontsize=14)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black', facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, zorder=0, edgecolor='black', facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE, zorder=1)
ax.add_feature(cfeature.BORDERS, zorder=1)
ax.add_feature(cfeature.LAKES, zorder=1, edgecolor='black', facecolor='lightblue')

estados_brasil.boundary.plot(ax=ax, edgecolor='black', linewidth=1, zorder=2)
gdf_clusters_ativos.boundary.plot(ax=ax, edgecolor='red', linewidth=1, zorder=4, label='Clusters Convectivos (T ≤ 235K)')
gdf_municipios_ativos.boundary.plot(ax=ax, edgecolor='blue', linewidth=1, zorder=3, label='Municípios Afetados')

# Adiciona legendas
plt.legend(loc='upper right')
plt.show()

### 9. Conclusão e Próximos Passos

**Parabéns!** Você concluiu com sucesso o segundo exemplo do pyfortracc. Neste tutorial, você aprendeu:

- **Instalação de dependências** adicionais (goesgcp, geobr)
- **Download de dados reais** do satélite GOES-19
- **Processamento de dados de infravermelho** para rastreamento de sistemas convectivos
- **Configuração de parâmetros avançados** incluindo correções vetoriais
- **Análise de sistemas meteorológicos reais** usando dados de temperatura de brilho
- **Visualização de trajetórias** de sistemas convectivos sobre o Brasil
- **Análise espacial com DuckDB** para identificar municípios com SCMs ativos
- **Criação de mapas cartográficos** mostrando sistemas e municípios impactados

### Recursos Adicionais

- [Documentação Oficial](https://fortracc.github.io/pyfortracc/)
- [Repositório GitHub](https://github.com/fortracc/pyfortracc)
- [Suporte](mailto:fortracc.project@inpe.br)
- [Artigo sobre Métodos Vetoriais](https://doi.org/10.3390/rs14215408)

### Dicas para Aplicações Operacionais

Para usar este exemplo em aplicações operacionais:

1. **Automatize o download**: Configure scripts para baixar dados regularmente
2. **Ajuste parâmetros**: Adapte thresholds e áreas mínimas para sua região de interesse
3. **Monitore performance**: Otimize para processamento em tempo real
4. **Valide resultados**: Compare com observações meteorológicas locais
5. **Integre com outros dados**: Combine com radar, modelos numéricos, etc.
6. **Configure alertas**: Use análise espacial para emitir avisos automáticos
7. **Desenvolva APIs**: Crie serviços web para disponibilizar resultados em tempo real